In [1]:
import advent

In [2]:
def get(op, code, pointer, offset):
    # e.g. get('1002', code, 2) -> code[p] (immediate mode)
    # e.g. get('2', code, 2) -> code[code[p]] (parameter mode)
    mode = op % (10**(2+offset)) // (10**(1+offset))
    #print(f"{op}, {pointer}, {offset}, {mode}, {code[pointer+offset]}")
    if mode == 0: # parameter mode
        return code[code[pointer+offset]]
    elif mode == 1: # immediate mode
        return code[pointer+offset]
    raise ValueError(f"{op}, {pointer}, {offset} not allowed")

class IO():
    # Simple implementation of IO: just has static input in the __init__

    def __init__(self, input_):
        self.input_buffer = input_.copy()
        self.output_buffer = []

    def read(self):
        return self.input_buffer.pop(0)
    
    def write(self, value):
        self.output_buffer.append(value)
        return self
    
    def output(self):
        return self.output_buffer.copy()

def step(code, p, io):
    # INPLACE does a intcode step, and returns new state and new p
    # p = -1 means the program is done
    op = code[p]
    if op % 100 == 1:
        code[code[p+3]] = get(op, code, p, 1) + get(op, code, p, 2)
        return code, p+4, io
    elif op % 100 == 2:
        code[code[p+3]] = get(op, code, p, 1) * get(op, code, p, 2)
        return code, p+4, io
    elif op % 100 == 3:
        code[code[p+1]] = io.read()
        return code, p+2, io
    elif op % 100 == 4:
        return code, p+2, io.write(get(op, code, p, 1))
    elif op % 100 == 5:
        if get(op, code, p, 1) != 0:
            return code, get(op, code, p, 2), io
        return code, p+3, io
    elif op % 100 == 6:
        if get(op, code, p, 1) == 0:
            return code, get(op, code, p, 2), io
        return code, p+3, io
    elif op % 100 == 7:
        code[code[p+3]] = int(get(op, code, p, 1) < get(op, code, p, 2))
        return code, p+4, io
    elif op % 100 == 8:
        code[code[p+3]] = int(get(op, code, p, 1) == get(op, code, p, 2))
        return code, p+4, io
    elif op % 100 == 99:
        return code, -1, io
    raise ValueError(f"Incorrect program. Op is {op}")

def run(code, pointer, io):
    while pointer >= 0:
        code, pointer, io = step(code, pointer, io)
    return code, io.output()

data = advent.get_intcode(5)
run(data, 0, IO([1]))[1]


[0, 0, 0, 0, 0, 0, 0, 0, 0, 16348437]

In [3]:
data = advent.get_intcode(5)
run(data, 0, IO([5]))[1]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 16348437, 6959377]